In [1]:
%matplotlib inline
# %matplotlib notebook
!git pull
# import os;os.environ['LAMBDA_PARQUET_TICK_DB']=rf'X:\lambda_data'
import os;os.environ['LAMBDA_PARQUET_TICK_DB']=rf'X:'

Already up to date.


In [2]:
from notebooks.lambda_imports import *
import time
import datetime
import tqdm
import seaborn as sns
sns.set_theme()

# from hurst import compute_Hc, random_walk
# plot_with_dark_style()

PROJECT_PATH=E:\javif\Coding\Python\market_making_fw\python_lambda
BACKTEST_OUTPUT_PATH(LAMBDA_OUTPUT_PATH)=X:\output_models
BACKTEST_TEMP_PATH(LAMBDA_TEMP_PATH)=E:\javif\Coding\Python\market_making_fw\java\temp
BACKTEST_JAR_PATH(LAMBDA_JAR_PATH)=E:\javif\Coding\Python\market_making_fw\java\executables\Backtest\target\Backtest.jar
log_path(LAMBDA_LOGS_PATH)=E:\javif\Coding\Python\market_making_fw\python_lambda\logs
PARQUET_PATH_DB(LAMBDA_PARQUET_TICK_DB)=X:


In [3]:
FIRST_HOUR=7.0
LAST_HOUR=19.0
tick = TickDB()
instrument_pk='btcusdt_binance'

start_dates=[]
end_dates=[]

start_date_fi=datetime.datetime(year=2020, day=7, month=12,hour=int(FIRST_HOUR))
end_date_fi=datetime.datetime(year=2020, day=7, month=12,hour=int(LAST_HOUR))

start_date_train=datetime.datetime(year=2020, day=8, month=12,hour=int(FIRST_HOUR))
end_date_train=datetime.datetime(year=2020, day=8, month=12,hour=int(LAST_HOUR))

start_dates=[start_date_fi,start_date_train]
end_dates=[end_date_fi,end_date_train]

start_date_tests=[]
end_date_tests=[]

days_test = [
    datetime.datetime(2020,12,9),
    datetime.datetime(2020,12,10),
    datetime.datetime(2020,12,11),
    datetime.datetime(2020,12,12),    
    datetime.datetime(2020,12,13),
    
    datetime.datetime(2020,12,14),
    datetime.datetime(2020,12,15),
    datetime.datetime(2020,12,16),    
    datetime.datetime(2020,12,17),
    datetime.datetime(2020,12,18),
    
    datetime.datetime(2020,12,19),
    datetime.datetime(2020,12,20),
    datetime.datetime(2020,12,21),
    datetime.datetime(2020,12,22),
    datetime.datetime(2020,12,23),
    
    datetime.datetime(2020,12,24),
    datetime.datetime(2020,12,25),
    datetime.datetime(2020,12,26),
    datetime.datetime(2020,12,27),
    datetime.datetime(2020,12,28),
    
    datetime.datetime(2020,12,29),
    datetime.datetime(2020,12,30),
    datetime.datetime(2021,1,1),
    datetime.datetime(2021,1,2),
    datetime.datetime(2021,1,3),
    
    datetime.datetime(2021,1,4),
    datetime.datetime(2021,1,5),
    datetime.datetime(2021,1,6),
    datetime.datetime(2021,1,7),
    datetime.datetime(2021,1,8),
    
]

for day in days_test:
    start_date = day+ datetime.timedelta(hours=int(FIRST_HOUR))
    end_date=day+ datetime.timedelta(hours=int(LAST_HOUR))
    start_date_tests.append(start_date)
    end_date_tests.append(end_date)
    
start_dates+=start_date_tests
end_dates+=end_date_tests

start_dates

[datetime.datetime(2020, 12, 7, 7, 0),
 datetime.datetime(2020, 12, 8, 7, 0),
 datetime.datetime(2020, 12, 9, 7, 0),
 datetime.datetime(2020, 12, 10, 7, 0),
 datetime.datetime(2020, 12, 11, 7, 0),
 datetime.datetime(2020, 12, 12, 7, 0),
 datetime.datetime(2020, 12, 13, 7, 0),
 datetime.datetime(2020, 12, 14, 7, 0),
 datetime.datetime(2020, 12, 15, 7, 0),
 datetime.datetime(2020, 12, 16, 7, 0),
 datetime.datetime(2020, 12, 17, 7, 0),
 datetime.datetime(2020, 12, 18, 7, 0),
 datetime.datetime(2020, 12, 19, 7, 0),
 datetime.datetime(2020, 12, 20, 7, 0),
 datetime.datetime(2020, 12, 21, 7, 0),
 datetime.datetime(2020, 12, 22, 7, 0),
 datetime.datetime(2020, 12, 23, 7, 0),
 datetime.datetime(2020, 12, 24, 7, 0),
 datetime.datetime(2020, 12, 25, 7, 0),
 datetime.datetime(2020, 12, 26, 7, 0),
 datetime.datetime(2020, 12, 27, 7, 0),
 datetime.datetime(2020, 12, 28, 7, 0),
 datetime.datetime(2020, 12, 29, 7, 0),
 datetime.datetime(2020, 12, 30, 7, 0),
 datetime.datetime(2021, 1, 1, 7, 0),
 date

In [4]:
#pt
depth_df=None
trade_df=None
bins=['5S','1T','1H']#5 secondss , 1 minute
df={}
ts_df={}
for i in range(len(start_dates)):
# for i in range(3):
    depth_df_it = tick.get_depth(instrument_pk=instrument_pk,start_date=start_dates[i],end_date=end_dates[i]).drop_duplicates()[start_dates[i]:end_dates[i]]
    trade_df_it = tick.get_trades(instrument_pk=instrument_pk,start_date=start_dates[i],end_date=end_dates[i]).drop_duplicates()[start_dates[i]:end_dates[i]]
    date = start_dates[i].replace(hour=0)
    
    df[date]=[depth_df_it,trade_df_it,len(depth_df_it),len(trade_df_it),len(depth_df_it)+len(trade_df_it)]
    for bin_it in bins:
        
        #ts tick counts
        ts = depth_df_it[depth_df_it.columns[0]].resample(bin_it).count()+trade_df_it[trade_df_it.columns[0]].resample(bin_it).count()
        ts_df_bin = pd.DataFrame(ts,columns=[date])
        
        #remove days from index
        ts_df_bin = ts_df_bin.reset_index()
        ts_df_bin['date'] = ts_df_bin['date'].apply(lambda x: x.strftime('%X'))
        ts_df_bin.set_index('date', inplace=True)
        
        if bin_it not in ts_df.keys():
            ts_df[bin_it]=ts_df_bin
        else:
             ts_df[bin_it][date]= ts_df_bin
        
        #summaries
        df[date]+=[
            depth_df_it[depth_df_it.columns[0]].resample(bin_it).count().mean(),
            trade_df_it[trade_df_it.columns[0]].resample(bin_it).count().mean(),
            depth_df_it[depth_df_it.columns[0]].resample(bin_it).count().mean()+ trade_df_it[trade_df_it.columns[0]].resample(bin_it).count().mean()
           ]
        
    if depth_df is None:
        depth_df=depth_df_it
    else:
        depth_df=depth_df.append(depth_df_it)
        
    if trade_df is None:
        trade_df=trade_df_it
    else:
        trade_df=trade_df.append(trade_df_it) 

downloading btcusdt_binance depth from 20201207 to 20201207
downloading btcusdt_binance trade from 20201207 to 20201207
downloading btcusdt_binance depth from 20201208 to 20201208
downloading btcusdt_binance trade from 20201208 to 20201208
downloading btcusdt_binance depth from 20201209 to 20201209
downloading btcusdt_binance trade from 20201209 to 20201209
downloading btcusdt_binance depth from 20201210 to 20201210
downloading btcusdt_binance trade from 20201210 to 20201210
downloading btcusdt_binance depth from 20201211 to 20201211
downloading btcusdt_binance trade from 20201211 to 20201211
downloading btcusdt_binance depth from 20201212 to 20201212
downloading btcusdt_binance trade from 20201212 to 20201212
downloading btcusdt_binance depth from 20201213 to 20201213
downloading btcusdt_binance trade from 20201213 to 20201213
downloading btcusdt_binance depth from 20201214 to 20201214
downloading btcusdt_binance trade from 20201214 to 20201214
downloading btcusdt_binance depth from 2

In [5]:
df = pd.DataFrame.from_dict(df).T
# df.to_csv('tick_db_backup.csv')

In [6]:
ts_df[bin_it]

,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
07:00:00,18459,21733,37777,18830,34232,22258,44822,18051,20965,23544,41856,40389,25407,37139,37909,38287,51241,34271,31238,24903,51964,39431,32656,47577,32564,35771,75421,85593,44969,55815,58769,61078
08:00:00,21223,32573,61797,21622,31679,25376,33581,19094,22925,21619,85549,50335,35168,43319,59033,49200,42716,37137,6439,29700,67482,49044,40323,68247,34465,33711,62166,72932,54326,63852,64333,53696
09:00:00,20820,49652,41875,28782,25926,23651,23528,21143,25135,23891,151023,37094,30247,41538,62644,38074,43097,39143,3579,30500,59576,37710,37536,57761,28239,38160,66550,91828,56700,68184,49761,59775
10:00:00,21796,41668,37916,30385,36234,21831,29688,23784,32651,55985,93390,41195,31366,41411,100770,37433,56654,35246,3592,32151,36177,43738,42028,37913,30146,30484,68645,141387,51636,54866,50527,105010
11:00:00,23361,30777,35996,24633,36022,22221,22772,19833,27076,46761,59395,44063,31069,28218,74159,46304,92105,34028,92183,28045,114461,38134,44291,42661,31396,20686,58254,94339,47303,80665,58471,98560
12:00:00,29788,41134,32346,22065,33754,20293,31958,22547,24827,46026,50564,41202,41769,39802,89304,46578,81334,33130,46325,28716,99908,44573,41112,35323,46682,103087,55362,88238,58819,68330,74153,94233
13:00:00,22911,34323,32564,31625,28359,18356,28708,22398,21568,72417,60961,43019,43619,48980,25464,43797,85996,41627,34790,30250,55249,60550,29835,40369,33156,86213,56674,59892,6989,64682,71297,66065
14:00:00,26753,28215,34085,30576,29730,27157,21622,30066,32160,117266,60690,58555,88174,37624,0,46927,72038,26145,51793,60666,67126,37160,32189,36028,39301,67810,70868,70531,56670,75871,78080,58274
15:00:00,22043,26970,32065,37293,26848,24284,34765,33089,26797,58499,57964,51492,62509,48087,0,41150,62041,31889,47895,59068,63929,44342,36566,45384,43614,57456,80439,62555,61294,56490,69998,120394


Por otro lado, creo que no caracterizamos en ninguna parte el flujo de ticks que se da en los datos de prueba 
(cuántos ticks se producen de media en 5 segundos, cuánto varía esa media durante un día de test, durante los 30 días, …). Creo que convendría hacerlo. (A lo mejor incluso nos da una pista de por qué hay un día en el que los Alpha-AS tienen malos resultados. el 21 diciembre)


In [7]:
# df=pd.read_csv('tick_db_backup.csv',index_col=0)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
2020-12-07,askPrice0 askPrice1 ...,price quantity da...,41244,254949,296193,4.77361,29.508,34.2816,57.2833,354.096,411.379,3437,21245.8,24682.8
2020-12-08,askPrice0 askPrice1 ...,price quantity da...,41882,339770,381652,4.84745,39.3252,44.1727,58.1694,471.903,530.072,3490.17,28314.2,31804.3
2020-12-09,askPrice0 askPrice1 ...,price quantity da...,42485,382755,425240,4.91725,44.3003,49.2176,59.0069,531.604,590.611,3540.42,31896.2,35436.7
2020-12-10,askPrice0 askPrice1 ...,price quantity da...,42209,276986,319195,4.8853,32.0586,36.9439,58.6236,384.703,443.326,3517.42,23082.2,26599.6
2020-12-11,askPrice0 askPrice1 ...,price quantity da...,42195,311927,354122,4.88368,36.1027,40.9863,58.6042,433.232,491.836,3516.25,25993.9,29510.2
2020-12-12,askPrice0 askPrice1 ...,price quantity da...,40062,245256,285318,4.63681,28.3861,33.0229,55.6417,340.633,396.275,3338.5,20438,23776.5
2020-12-13,askPrice0 askPrice1 ...,price quantity da...,42262,317149,359411,4.89144,36.7071,41.5985,58.6972,440.485,499.182,3521.83,26429.1,29950.9
2020-12-14,askPrice0 askPrice1 ...,price quantity da...,39674,243754,283428,4.5919,28.2123,32.8042,55.1028,338.547,393.65,3306.17,20312.8,23619
2020-12-15,askPrice0 askPrice1 ...,price quantity da...,40131,274341,314472,4.64479,31.7524,36.3972,55.7375,381.029,436.767,3344.25,22861.8,26206
2020-12-16,askPrice0 askPrice1 ...,price quantity da...,41591,560151,601742,4.81377,64.8323,69.6461,57.7653,777.987,835.753,3465.92,46679.2,50145.2


In [8]:
summary_df=df.loc[:,2:]
columns_rename = {2:"count_depth",3:"count_trades",4:"count_ticks"}
summary_df=summary_df.rename(columns=columns_rename)
iterator=int(list(columns_rename.keys())[-1])
for bin_it in bins:
#     summary_df=summary_df.rename(columns={iterator:f"df_count_depth_{bin_it}",iterator+1:f"df_count_trade_{bin_it}",iterator+2:f"df_count_ticks_{bin_it}"})
    summary_df=summary_df.rename(columns={iterator+1:f"count_depth_{bin_it}",iterator+2:f"count_trade_{bin_it}",iterator+3:f"count_ticks_{bin_it}"})
    iterator+=3
# summary_df["ticks_average_per_second"]=(summary_df["count_depth"]+summary_df["count_trades"])/((LAST_HOUR-FIRST_HOUR)*60*60)
# summary_df["ticks_average_per_5seconds"]=summary_df["ticks_average_per_second"]*5
# summary_df.fillna(summary_df.mean(),inplace=True)
summary_df
#### 21 dec is the worst dd day

,count_depth,count_trades,count_ticks,count_depth_5S,count_trade_5S,count_ticks_5S,count_depth_1T,count_trade_1T,count_ticks_1T,count_depth_1H,count_trade_1H,count_ticks_1H
2020-12-07,41244,254949,296193,4.77361,29.508,34.2816,57.2833,354.096,411.379,3437,21245.8,24682.8
2020-12-08,41882,339770,381652,4.84745,39.3252,44.1727,58.1694,471.903,530.072,3490.17,28314.2,31804.3
2020-12-09,42485,382755,425240,4.91725,44.3003,49.2176,59.0069,531.604,590.611,3540.42,31896.2,35436.7
2020-12-10,42209,276986,319195,4.8853,32.0586,36.9439,58.6236,384.703,443.326,3517.42,23082.2,26599.6
2020-12-11,42195,311927,354122,4.88368,36.1027,40.9863,58.6042,433.232,491.836,3516.25,25993.9,29510.2
2020-12-12,40062,245256,285318,4.63681,28.3861,33.0229,55.6417,340.633,396.275,3338.5,20438,23776.5
2020-12-13,42262,317149,359411,4.89144,36.7071,41.5985,58.6972,440.485,499.182,3521.83,26429.1,29950.9
2020-12-14,39674,243754,283428,4.5919,28.2123,32.8042,55.1028,338.547,393.65,3306.17,20312.8,23619
2020-12-15,40131,274341,314472,4.64479,31.7524,36.3972,55.7375,381.029,436.767,3344.25,22861.8,26206
2020-12-16,41591,560151,601742,4.81377,64.8323,69.6461,57.7653,777.987,835.753,3465.92,46679.2,50145.2


In [9]:
columns=['count_ticks']
for bin_it in bins:
    columns.append(rf"count_ticks_{bin_it}")
summary_ticks_df=summary_df[columns]
summary_ticks_df

,count_ticks,count_ticks_5S,count_ticks_1T,count_ticks_1H
2020-12-07,296193,34.2816,411.379,24682.8
2020-12-08,381652,44.1727,530.072,31804.3
2020-12-09,425240,49.2176,590.611,35436.7
2020-12-10,319195,36.9439,443.326,26599.6
2020-12-11,354122,40.9863,491.836,29510.2
2020-12-12,285318,33.0229,396.275,23776.5
2020-12-13,359411,41.5985,499.182,29950.9
2020-12-14,283428,32.8042,393.65,23619
2020-12-15,314472,36.3972,436.767,26206
2020-12-16,601742,69.6461,835.753,50145.2


In [10]:
summary_df.to_csv("ticks_article.csv")

# TimeSeries analysis

In [11]:
for bin_it in bins:
    ts_df[bin_it].to_csv(rf"ticks_article_ts_{bin_it}.csv")

In [ ]:
#5 seconds
# ax=plt.figure(15,12)
index_bin='5S'

ax = ts_df[index_bin].plot.bar(rot=0,figsize=(15,12))
plt.xlabel('time bin')
plt.ylabel('tick count')
plt.title(index_bin)
plt.legend(loc='best')